In [1]:
import pandas as pd
import numpy  as np
import time
from sklearn.ensemble import RandomForestRegressor
import RFfeatureEval
import pickle
from setPandasOptions import setPandasOptions
from getConfig import getConfig
from getData import getData
import prepData
from calcRMSE import calcRMSE
from getRFparms import getParms
from selectSet import selectSet
from getSet import getSet
#setPandasOptions()

In [2]:
config = getConfig()

In [3]:
def saveModel(model, count):
    fname = "RFmodel_" + str(count)
    pickle.dump(model, open(config["modelDir"] + fname, 'wb'))

In [5]:
Set = selectSet()

Enter a number from 1-41

Processing set  1


In [6]:
train, test = getData(config)
train = getSet(train, Set)
test = getSet(test, Set)

dataDict = prepData.process(train, test, config)

In [7]:
'''# Figure out if you need all the features
regr = RandomForestRegressor(n_estimators=100)
rf   = regr.fit(data["trainX"], data["trainY"])

cols = RFfeatureEval.process(data["trainX"].columns, rf)

keep  = [col[0] for col in cols if col[1] > .01]    # discard non-useful features
data["trainX"] = data["trainX"][keep]
data["testX"]  = data["testX"][keep]'''

'# Figure out if you need all the features\nregr = RandomForestRegressor(n_estimators=100)\nrf   = regr.fit(data["trainX"], data["trainY"])\n\ncols = RFfeatureEval.process(data["trainX"].columns, rf)\n\nkeep  = [col[0] for col in cols if col[1] > .01]    # discard non-useful features\ndata["trainX"] = data["trainX"][keep]\ndata["testX"]  = data["testX"][keep]'

In [8]:
parms = getParms()

results = []
count = 1

for x in parms:
    trees     = x[0]
    nodeSize = x[1]
    depth     = x[2]
    leafSize = x[3]
    features  = x[4]
    regr = RandomForestRegressor(n_estimators = trees,\
                                 min_samples_split = nodeSize,\
                                 max_depth = depth,\
                                 min_samples_leaf = leafSize,\
                                 max_features = features)

    rf    = regr.fit(dataDict["trainX"], dataDict["trainY"])
    preds = rf.predict(dataDict["testX"])
    score = calcRMSE(dataDict["testY"], preds)
    tmp = (count, x, score)
    results.append(tmp)
    saveModel(rf, count)
    count += 1
    print("Done with {} of {}".format(count, len(parms)))

Done with 1 of 1


In [7]:
with open("/home/tbrownex/modelResults.csv", "w") as f:
    hdr = "count"+"|"+"trees"+"|"+"nodes"+"|"+"depth"+"|"+\
    "leafs"+"|"+"features"+"|"+"score"+"\n"
    f.write(hdr)
    for x in results:
        count = x[0]
        parms = x[1]
        score = x[2]
        trees = parms[0]
        nodes = parms[1]
        depth = parms[2]
        leafs = parms[3]
        features = parms[4]
        rec = str(count)+"|"+str(trees)+"|"+str(nodes)+"|"+str(depth)+\
        "|"+str(leafs)+"|"+str(features)+"|"+str(round(score,2))+"\n"
        f.write(rec)